# **Customer Lookalike Model: Similarity-Based Recommendations**

In [16]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import ast

In [2]:
customers = pd.read_csv('Customers.csv')
customers.head(2)

,CustomerID,CustomerName,Region,SignupDate
0,C0001,Lawrence Carroll,South America,2022-07-10
1,C0002,Elizabeth Lutz,Asia,2022-02-13


In [3]:
products = pd.read_csv('Products.csv')
products.head(2)

,ProductID,ProductName,Category,Price
0,P001,ActiveWear Biography,Books,169.3
1,P002,ActiveWear Smartwatch,Electronics,346.3


In [4]:
transactions = pd.read_csv('Transactions.csv')
transactions.head(2)

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68


## Creating a Copy of the Dataset  
- Ensuring the original data remains unchanged during analysis.

In [5]:
customers_copy = customers.copy()
products_copy = products.copy()
transactions_copy = transactions.copy()

## **Step 1: Data Preparation **

### **Why These Steps?**
✅ **Ensure all customer attributes are available for similarity comparison.**  
✅ **Avoid bias due to differing feature scales.**  
✅ **Prepare structured and meaningful numerical data for similarity calculations.**  

The dataset is now **fully prepared** for similarity computation in the next step.


In [ ]:
transactions_with_products = transactions_copy.merge(products_copy, on="ProductID")
customer_summary = transactions_with_products.groupby("CustomerID").agg(
    Total_Spending=("TotalValue", "sum"),
    Purchase_Count=("TransactionID", "count"),
    Unique_Products=("ProductID", "nunique"),
).reset_index()

customer_product_prefs = transactions_with_products.groupby(["CustomerID", "Category"])["TransactionID"].count().unstack(fill_value=0)
customer_features = customers_copy.merge(customer_summary, on="CustomerID", how="left").fillna(0)
customer_features = customer_features.merge(customer_product_prefs, on="CustomerID", how="left").fillna(0)
customer_features = pd.get_dummies(customer_features, columns=["Region"], drop_first=True)

In [8]:
scaler = MinMaxScaler()
num_cols = ["Total_Spending", "Purchase_Count", "Unique_Products"] + list(customer_product_prefs.columns)
customer_features[num_cols] = scaler.fit_transform(customer_features[num_cols])
customer_features.set_index("CustomerID", inplace=True)
customer_features.head()

,CustomerName,SignupDate,Total_Spending,Purchase_Count,Unique_Products,Books,Clothing,Electronics,Home Decor,Region_Europe,Region_North America,Region_South America
CustomerID,,,,,,,,,,,,
C0001,Lawrence Carroll,2022-07-10,0.314274,0.454545,0.5,0.2,0.0,0.6,0.166667,False,False,True
C0002,Elizabeth Lutz,2022-02-13,0.174514,0.363636,0.4,0.0,0.4,0.0,0.333333,False,False,False
C0003,Michael Rivera,2024-03-07,0.255332,0.363636,0.4,0.0,0.2,0.2,0.333333,False,False,True
C0004,Kathleen Rodriguez,2022-10-09,0.501681,0.727273,0.8,0.6,0.0,0.4,0.500000,False,False,True
C0005,Laura Weber,2022-08-15,0.190581,0.272727,0.3,0.0,0.0,0.4,0.166667,False,False,False


## Step 2: Similarity Computation and Lookalike Extraction

- **Constructed a Feature Matrix** by removing non-numeric columns (`CustomerName`, `SignupDate`) to ensure accurate similarity computation.
- **Computed the Similarity Matrix** using **Cosine Similarity**, which measures how similar customers are based on their shopping behavior.
- **Defined a Function (`get_top_similar`)** to:
  - Retrieve the top 3 most similar customers for each given customer.
  - Exclude self-comparison to avoid a 100% similarity match.
  - Return a dictionary mapping each customer to their most similar customers and similarity scores.
- **Generated a Lookalike Mapping** for the first 20 customers and stored it in `Lookalike.csv` for further analysis.


In [14]:
customer_feature_matrix = customer_features.drop(columns=["CustomerName", "SignupDate"], errors="ignore")
similarity_matrix = pd.DataFrame(cosine_similarity(customer_feature_matrix),index=customer_feature_matrix.index,columns=customer_feature_matrix.index)

def get_top_similar(customers, top_n=3):
    similar_customers = {}
    for cust_id in customers:
        top_similar = similarity_matrix.loc[cust_id].drop(cust_id).nlargest(top_n)
        similar_customers[cust_id] = list(zip(top_similar.index, top_similar.values))
    
    return similar_customers
top_lookalikes = get_top_similar(customer_feature_matrix.index[:20])
lookalike_df = pd.DataFrame([
    {"CustomerID": cust_id, "Similar_Customers": str(similar)} 
    for cust_id, similar in top_lookalikes.items()
])

lookalike_df.to_csv("Lookalike.csv", index=False)
lookalike_df.head(5)


,CustomerID,Similar_Customers
0,C0001,"[('C0190', 0.9796638286783601), ('C0048', 0.97..."
1,C0002,"[('C0134', 0.9911380588628481), ('C0159', 0.96..."
2,C0003,"[('C0031', 0.9985639550443514), ('C0158', 0.99..."
3,C0004,"[('C0113', 0.9863757421367481), ('C0012', 0.97..."
4,C0005,"[('C0007', 0.9971116579101608), ('C0140', 0.97..."


# Model Validation

In [23]:
lookalike_df = pd.read_csv("Lookalike.csv")
lookalike_df["Similar_Customers"] = lookalike_df["Similar_Customers"].apply(ast.literal_eval)

selected_customer = "C0001"
top_similar_customers = [cust[0] for cust in lookalike_df.loc[lookalike_df["CustomerID"] == selected_customer, "Similar_Customers"].values[0]]
features_to_compare = ["Total_Spending", "Purchase_Count", "Unique_Products"] + list(customer_feature_matrix.columns[3:])  # Including category preferences
comparison_df = customer_features.loc[[selected_customer] + top_similar_customers, features_to_compare]
comparison_df

,Total_Spending,Purchase_Count,Unique_Products,Books,Clothing,Electronics,Home Decor,Region_Europe,Region_North America,Region_South America
CustomerID,,,,,,,,,,
C0001,0.314274,0.454545,0.5,0.2,0.0,0.6,0.166667,False,False,True
C0190,0.279469,0.454545,0.5,0.2,0.2,0.4,0.166667,False,False,True
C0048,0.360782,0.454545,0.5,0.2,0.2,0.4,0.166667,False,False,True
C0091,0.293957,0.545455,0.5,0.0,0.2,0.8,0.166667,False,False,True


In [44]:
selected_customer = "C0007"
top_similar_customers = [cust[0] for cust in lookalike_df.loc[lookalike_df["CustomerID"] == selected_customer, "Similar_Customers"].values[0]]
features_to_compare = ["Total_Spending", "Purchase_Count", "Unique_Products"] + list(customer_feature_matrix.columns[3:])  # Including category preferences
comparison_df = customer_features.loc[[selected_customer] + top_similar_customers, features_to_compare]
comparison_df

,Total_Spending,Purchase_Count,Unique_Products,Books,Clothing,Electronics,Home Decor,Region_Europe,Region_North America,Region_South America
CustomerID,,,,,,,,,,
C0007,0.241695,0.272727,0.3,0.0,0.0,0.4,0.166667,False,False,False
C0005,0.190581,0.272727,0.3,0.0,0.0,0.4,0.166667,False,False,False
C0140,0.176894,0.181818,0.2,0.0,0.0,0.2,0.166667,False,False,False
C0045,0.564163,0.636364,0.7,0.2,0.2,0.8,0.166667,False,False,False


## **Model Validation Interpretation**

The validation process demonstrates that the lookalike model effectively identifies customers with similar purchase behaviors. 

### **Spending and Purchases**
The Total_Spending and Purchase_Count values for similar customers closely align with the selected customer, indicating a strong correlation in purchasing habits. For instance, in the case of **C0001**, its similar customers **C0190, C0048, and C0091** have spending values that are close, reinforcing the model's accuracy in identifying behavioral similarities.

### **Product Preferences**
The product category preferences for books remain consistent across similar customers, confirming a shared interest. Minor differences in electronics and clothing preferences suggest variations in buying behavior but remain within an acceptable range of similarity.

### **Regional Influence**
All similar customers belong to the same region as the selected customer, ensuring that the model considers regional preferences while making recommendations.

### **Conclusion**
The lookalike model is successful in grouping customers based on spending behavior, purchase frequency, product interests, and regional attributes. While some variations exist, the model effectively captures meaningful similarities, making it a reliable tool for customer segmentation and targeted marketing strategies.
